# ELF-based Charge Analysis Example

This notebook demonstrates an ELF-based charge analysis method for counting electrons in electrides.

**Reference:**
Weaver et al., "Counting Electrons in Electrides", J. Am. Chem. Soc. 2023, 145, 26472-26476

## Features
- Used the algorithm for identifying the electride sites, same as the reference.
- Simple Voronoi partitioning: Stable and robust partitioning method, not the same as the reference

## Two Ways to Use elfcharge

1. **Easy-Use Wrapper** (`run_badelf_analysis`): One function call for complete analysis
2. **Step-by-Step API**: Fine-grained control over each step

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.insert(0, '..')

# Import elfcharge modules
from elfcharge import (
    read_elfcar, read_chgcar,
    ELFAnalyzer,
    VoronoiPartitioner,
    create_structure_with_electrides,
    DEFAULT_ZVAL  # Materials Project recommended POTCAR values
)
from elfcharge.visualize import (
    plot_elf_profiles_by_type,
    plot_radial_elf,
    plot_electron_distribution,
    plot_electride_elf_histogram,
    plot_elf_slice,
    plot_elf_slice_with_partition  # New: partition visualization
)
from scipy.ndimage import zoom

## Option A: Easy-Use Wrapper (Recommended)

Use `run_badelf_analysis()` for a complete analysis in one function call.

In [ ]:
import sys
sys.path.insert(0, '..')

from elfcharge import run_badelf_analysis, DEFAULT_ZVAL

# Show some DEFAULT_ZVAL values
print("DEFAULT_ZVAL examples (Materials Project recommended POTCARs):")
for elem in ['Y', 'C', 'La', 'Mg', 'H']:
    if elem in DEFAULT_ZVAL:
        print(f"  {elem}: {DEFAULT_ZVAL[elem]}")

# Paths to VASP output files (Y2C electride example)
DATA_DIR = '../test/Y2C'

# Run complete BadELF analysis with one function call
# Note: Use CHGCAR (valence electrons only), not CHGCAR_sum
# DEFAULT_ZVAL is used automatically if zval is not specified
result = run_badelf_analysis(
    elfcar_path=f'{DATA_DIR}/ELFCAR',
    chgcar_path=f'{DATA_DIR}/CHGCAR',  # Valence electrons only!
    # Oxidation states for improved CrystalNN accuracy
    oxidation_states={'Y': 3, 'C': -4},
    # Core radii: exclude atomic core region when finding ELF minima along bonds
    # Important for heavy elements where ELF is low in the core
    core_radii={'Y': 0.8, 'C': 0.3},
    # Electride detection threshold
    elf_threshold=0.5,
    # Apply ELF smoothing for cleaner visualization
    apply_smooth=True,
    smooth_size=3,
    # Output options
    save_dir='./badelf_output',
    save_plots=True,
    save_cif=True,
    verbose=True
)

In [ ]:
# Access results from run_badelf_analysis
print("="*60)
print("Results from run_badelf_analysis()")
print("="*60)

# Species average oxidation states
print("\nSpecies average oxidation states:")
for species, ox in result.oxidation.species_avg_oxidation.items():
    print(f"  {species}: {ox:+.3f}")

# Electride site information
print(f"\nElectride sites found: {len(result.electride_sites)}")
if len(result.electride_sites) > 0:
    # Note: All electride sites are merged into a single region
    total_e_electrons = np.sum(result.charges.electride_electrons)
    print(f"  Total electride electrons: {total_e_electrons:.3f}")
    print(f"  (merged from {len(result.electride_sites)} detected sites)")

# Total electrons
print(f"\nTotal electrons: {result.total_electrons:.4f}")

# Available output files
print("\nOutput files saved to ./badelf_output/:")
print("  - structure_with_electrides.cif")
print("  - elf_partition_xy.png")
print("  - electron_distribution.png")

## Option B: Step-by-Step API

For more control over the analysis process, use the individual functions.
This approach is useful when you need to customize specific steps or inspect intermediate results.

In [ ]:
# Paths to VASP output files (Y2C electride)
DATA_DIR = '../test/Y2C'
ELFCAR_PATH = f'{DATA_DIR}/ELFCAR'
CHGCAR_PATH = f'{DATA_DIR}/CHGCAR'  # Valence electrons only

# Load data
print("Loading ELFCAR...")
elf_data = read_elfcar(ELFCAR_PATH)

print("Loading CHGCAR...")
chg_data = read_chgcar(CHGCAR_PATH)

print(f"\nStructure info:")
print(f"  Volume: {elf_data.volume:.2f} Å³")
print(f"  Atoms: {elf_data.n_atoms}")
print(f"  Species: {dict(zip(elf_data.species, elf_data.num_atoms))}")
print(f"  ELF grid: {elf_data.ngrid}")
print(f"  CHGCAR grid: {chg_data.ngrid}")

# Check total electrons
total_electrons = np.sum(chg_data.grid) / np.prod(chg_data.ngrid)
print(f"\nTotal electrons in CHGCAR: {total_electrons:.2f}")

## 2. ZVAL Configuration

The package includes DEFAULT_ZVAL based on Materials Project recommended POTCARs.
You can use it directly or override with custom values.

In [ ]:
# DEFAULT_ZVAL from Materials Project recommended POTCARs
from elfcharge import DEFAULT_ZVAL

# Show values for elements in our system (Y2C)
print("DEFAULT_ZVAL values for Y2C:")
for sp in elf_data.species:
    if sp in DEFAULT_ZVAL:
        print(f"  {sp}: {DEFAULT_ZVAL[sp]}")

# Use DEFAULT_ZVAL directly
ZVAL_CONFIG = {sp: DEFAULT_ZVAL.get(sp, 0) for sp in elf_data.species}
print(f"\nZVAL_CONFIG: {ZVAL_CONFIG}")

# Expected total valence electrons
expected = sum(n * ZVAL_CONFIG.get(sp, 0) 
               for sp, n in zip(elf_data.species, elf_data.num_atoms))
print(f"Expected valence electrons: {expected}")
print(f"Actual: {total_electrons:.2f}")

## 3. Find Electride Sites

Electride sites are interstitial ELF maxima that are:
- Far from all atomic nuclei
- Have high ELF values (> threshold)

In [ ]:
# Initialize analyzer
analyzer = ELFAnalyzer(elf_data)

# First, analyze bonds to get elf_radii for electride detection
bonds = analyzer.get_neighbor_pairs_pymatgen(oxidation_states={'Y': 3, 'C': -4})
print(f"Found {len(bonds)} neighbor pairs")

# Analyze ELF along each bond
# core_radii: exclude atomic core regions when finding ELF minima
core_radii = {'Y': 0.8, 'C': 0.3}
for i, bond in enumerate(bonds):
    bonds[i] = analyzer.analyze_bond_elf(bond, core_radii=core_radii)

# Compute ELF radii from ELF minima along bonds
elf_radii = analyzer.compute_elf_radii(bonds)

# Find interstitial electride sites using elf_radii
# This uses the boundary_radius (max distance to ELF minimum) for detection
electride_sites = analyzer.find_interstitial_electrides(
    elf_threshold=0.5,
    elf_radii=elf_radii
)

print(f"Found {len(electride_sites)} electride sites")
print("\nElectride site details:")
for i, site in enumerate(sorted(electride_sites, key=lambda x: -x.elf_value)):
    print(f"  Site {i}: ELF = {site.elf_value:.3f}, position = {site.frac_coord}")

## 4. Analyze Bond ELF Profiles

Display ELF profiles along bonds and atomic radii.

In [ ]:
# Display bond analysis results
n_with_min = sum(1 for b in bonds if b.elf_minimum_frac is not None)
print(f"Bonds with ELF minima: {n_with_min}/{len(bonds)}")

# Show ELF radii from ELF minima
species_list = elf_data.get_species_list()
print("\nELF radii (from ELF minima):")
for sp in sorted(set(species_list)):
    indices = [i for i, s in enumerate(species_list) if s == sp]
    radii = [elf_radii[i].boundary_radius for i in indices 
             if elf_radii[i] and elf_radii[i].boundary_radius]
    if radii:
        print(f"  {sp}: {np.mean(radii):.3f} ± {np.std(radii):.3f} Å")

In [ ]:
# Visualize ELF profiles
fig = plot_elf_profiles_by_type(elf_data, bonds)
fig.savefig('elf_profiles.png', dpi=150)
print("Saved elf_profiles.png")

## 5. Spatial Partitioning

Partition space using simple Voronoi method:
- All sites (atoms + electrides) use simple Voronoi
- Electride sites are merged into a single region

In [ ]:
n_atoms = elf_data.n_atoms
n_electride = len(electride_sites)
ngx, ngy, ngz = elf_data.ngrid
lattice = elf_data.lattice

# Create coordinate list (atoms + electrides)
if n_electride > 0:
    all_frac_coords = np.vstack([
        elf_data.frac_coords,
        np.array([site.frac_coord for site in electride_sites])
    ])
else:
    all_frac_coords = elf_data.frac_coords

# Simple Voronoi partitioning
fx = np.linspace(0, 1, ngx, endpoint=False)
fy = np.linspace(0, 1, ngy, endpoint=False)
fz = np.linspace(0, 1, ngz, endpoint=False)
FX, FY, FZ = np.meshgrid(fx, fy, fz, indexing='ij')
grid_frac = np.stack([FX, FY, FZ], axis=-1)

min_dist_sq = np.full((ngx, ngy, ngz), np.inf)
labels = np.zeros((ngx, ngy, ngz), dtype=np.int32)

n_total = n_atoms + n_electride
for i in range(n_total):
    site_frac = all_frac_coords[i]
    diff = grid_frac - site_frac
    diff = diff - np.round(diff)  # Minimum image convention
    diff_cart = np.einsum('...j,jk->...k', diff, lattice)
    dist_sq = np.sum(diff_cart ** 2, axis=-1)
    
    closer = dist_sq < min_dist_sq
    labels[closer] = i
    min_dist_sq[closer] = dist_sq[closer]

# Create final labels: atoms 1 to n_atoms, all electrides merged as n_atoms + 1
final_labels = np.zeros((ngx, ngy, ngz), dtype=np.int32)
for i in range(n_atoms):
    final_labels[labels == i] = i + 1
if n_electride > 0:
    for i in range(n_atoms, n_total):
        final_labels[labels == i] = n_atoms + 1

print(f"Simple Voronoi partitioning complete")
print(f"  Atomic regions: {n_atoms}")
print(f"  Electride region: 1 (merged from {n_electride} sites)")

## 6. Charge Integration

In [ ]:
# Use final_labels for charge integration
labels_for_integration = final_labels

# Resample labels to CHGCAR grid if needed
if elf_data.ngrid != chg_data.ngrid:
    zoom_factors = tuple(t / s for t, s in zip(chg_data.ngrid, elf_data.ngrid))
    labels_resampled = zoom(labels_for_integration.astype(float), zoom_factors, order=0).astype(int)
else:
    labels_resampled = labels_for_integration

chg_grid = chg_data.grid
n_grid = np.prod(chg_data.ngrid)

# Integrate for atoms (labels 1 to n_atoms)
atom_electrons = np.zeros(n_atoms)
for i in range(n_atoms):
    mask = labels_resampled == (i + 1)
    atom_electrons[i] = np.sum(chg_grid[mask]) / n_grid

# Integrate for electride region (label n_atoms + 1)
if n_electride > 0:
    mask = labels_resampled == (n_atoms + 1)
    total_electride_electrons = np.sum(chg_grid[mask]) / n_grid
else:
    total_electride_electrons = 0.0

# Calculate oxidation states
atom_oxidation = np.zeros(n_atoms)
for i, sp in enumerate(species_list):
    ref = ZVAL_CONFIG.get(sp, 0)
    atom_oxidation[i] = ref - atom_electrons[i]

print(f"Total electrons: {np.sum(atom_electrons) + total_electride_electrons:.4f}")

In [ ]:
print("="*60)
print("RESULTS")
print("="*60)

print("\nSpecies average oxidation states:")
print("-"*50)
species_stats = {}
for sp in sorted(set(species_list)):
    indices = [i for i, s in enumerate(species_list) if s == sp]
    avg_e = np.mean([atom_electrons[i] for i in indices])
    std_e = np.std([atom_electrons[i] for i in indices])
    avg_ox = np.mean([atom_oxidation[i] for i in indices])
    std_ox = np.std([atom_oxidation[i] for i in indices])
    ref = ZVAL_CONFIG.get(sp, 0)
    species_stats[sp] = {'count': len(indices), 'mean_ox': avg_ox, 'ref': ref}
    print(f"  {sp:3s}: {avg_ox:+6.3f} ± {std_ox:.3f}  (electrons: {avg_e:.2f} ± {std_e:.2f}, ZVAL={ref})")

print("\nElectride region:")
print("-"*50)
print(f"  Total electride electrons: {total_electride_electrons:.4f}")
print(f"  (from {n_electride} detected sites, merged into single region)")

print("="*60)
print("RESULTS")
print("="*60)

print("\nSpecies average oxidation states:")
print("-"*50)
species_stats = {}
for sp in sorted(set(species_list)):
    indices = [i for i, s in enumerate(species_list) if s == sp]
    avg_e = np.mean([atom_electrons[i] for i in indices])
    std_e = np.std([atom_electrons[i] for i in indices])
    avg_ox = np.mean([atom_oxidation[i] for i in indices])
    std_ox = np.std([atom_oxidation[i] for i in indices])
    ref = ZVAL_CONFIG.get(sp, 0)
    species_stats[sp] = {'count': len(indices), 'mean_ox': avg_ox, 'ref': ref}
    print(f"  {sp:3s}: {avg_ox:+6.3f} ± {std_ox:.3f}  (electrons: {avg_e:.2f} ± {std_e:.2f}, ZVAL={ref})")

print("\nElectride region:")
print("-"*50)
print(f"  Total electride electrons: {total_electride_electrons:.4f}")
print(f"  (from {n_electride} detected sites, merged into single region)")

# Charge neutrality check
print("\nCharge neutrality check:")
print("-"*50)
total_charge = 0.0
print("  Contribution by species:")
for sp, stats in species_stats.items():
    contrib = stats['count'] * stats['mean_ox']
    total_charge += contrib
    print(f"    {sp}: {stats['count']} × ({stats['mean_ox']:+.3f}) = {contrib:+.3f}")

electride_charge = -total_electride_electrons
print(f"    Electride: {electride_charge:+.3f}")
total_charge += electride_charge

print(f"\n  Total charge (should be 0): {total_charge:+.4f}")

# Create structure with electride sites as dummy atoms
structure = create_structure_with_electrides(
    elf_data,
    electride_sites,
    electride_symbol="X"  # Dummy atom
)

print(f"Structure created with {structure.num_sites} sites")
print(f"  Atoms: {n_atoms}")
print(f"  Electride sites: {n_electride}")

# Export to CIF
structure.to(filename="structure_with_electrides.cif", fmt="cif")
print("\nExported to structure_with_electrides.cif")

In [ ]:
# Plot ELF slice with partition boundaries (Cartesian coordinates, interpolated)
fig = plot_elf_slice_with_partition(
    elf_data, final_labels,
    plane='xy', position=0.5,
    electride_sites=electride_sites,
    # High resolution with interpolation
    interpolate=True,
    interpolate_factor=3,
    # Coordinate system: 'cartesian' (Å) or 'fractional'
    coord_system='cartesian',
    # Atom display options
    show_atoms=True,
    show_atom_symbol=True,
    atom_symbol='o',
    atom_symbol_size=200,
    atom_symbol_color={'Y': '#4169E1', 'C': '#32CD32'},
    # Element label options
    show_element_label=True,
    element_label_size=10,
    element_label_color='black',
    # Electride display options
    show_electrides=True,
    electride_symbol='*',
    electride_symbol_size=300,
    electride_symbol_color='yellow',
    electride_label='e⁻',
    electride_label_size=10,
    electride_label_color='black',
)
fig.savefig('elf_partition_cartesian.png', dpi=150)
plt.show()
print("Saved elf_partition_cartesian.png")

In [ ]:
# Create structure with electride sites as dummy atoms
structure = create_structure_with_electrides(
    elf_data,
    electride_sites,
    electride_symbol="X"  # Dummy atom
)

print(f"Structure created with {structure.num_sites} sites")
print(f"  Atoms: {n_atoms}")
print(f"  Electride sites: {n_electride}")

# Export to CIF
structure.to(filename="structure_with_electrides.cif", fmt="cif")
print("\nExported to structure_with_electrides.cif")

# Different planes (xz, yz)
# XZ plane
fig_xz = plot_elf_slice_with_partition(
    elf_data, final_labels,
    plane='xz', position=0.5,
    electride_sites=electride_sites,
    interpolate=True,
    interpolate_factor=2,
    coord_system='cartesian',
)
fig_xz.savefig('elf_partition_xz.png', dpi=150)
plt.show()

# YZ plane
fig_yz = plot_elf_slice_with_partition(
    elf_data, final_labels,
    plane='yz', position=0.5,
    electride_sites=electride_sites,
    interpolate=True,
    interpolate_factor=2,
    coord_system='cartesian',
)
fig_yz.savefig('elf_partition_yz.png', dpi=150)
plt.show()

print("Saved elf_partition_xz.png and elf_partition_yz.png")

In [ ]:
# Plot radial ELF around La atom
fig, ax = plt.subplots(figsize=(8, 5))
la_indices = [i for i, s in enumerate(species_list) if s == 'La']
plot_radial_elf(elf_data, la_indices[0], r_max=3.0, ax=ax)
fig.savefig('la_radial_elf.png', dpi=150)
plt.show()
print("Saved la_radial_elf.png")

In [ ]:
# Plot radial ELF around Y atom
fig, ax = plt.subplots(figsize=(8, 5))
y_indices = [i for i, s in enumerate(species_list) if s == 'Y']
if y_indices:
    plot_radial_elf(elf_data, y_indices[0], r_max=3.0, ax=ax)
    fig.savefig('y_radial_elf.png', dpi=150)
    plt.show()
    print("Saved y_radial_elf.png")

In [ ]:
# Plot electride site statistics
fig = plot_electride_elf_histogram(electride_sites, electride_electrons)
fig.savefig('electride_stats.png', dpi=150)
print("Saved electride_stats.png")

## Summary

This notebook demonstrated:

1. **Loading VASP output files** (ELFCAR, CHGCAR)
2. **Finding electride sites** from interstitial ELF maxima
3. **Analyzing bond ELF profiles** to find atomic radii
4. **Spatial partitioning** using simple Voronoi
5. **Charge integration** for atoms and electride sites
6. **Oxidation state calculation** using DEFAULT_ZVAL from Materials Project
7. **Structure export** with electride sites as dummy atoms
8. **Visualization** of ELF and electron distributions

### Key Features

- **DEFAULT_ZVAL**: Built-in valence electron counts based on Materials Project recommended POTCARs (~80 elements)
- **Simple Voronoi partitioning**: Stable and robust method for all systems
- **Merged electride regions**: All detected electride sites are merged into a single region for charge integration

### Parameter Guide

- **core_radii**: Used when finding ELF minima along bonds. Excludes atomic core region where ELF is low (important for heavy elements like Y, La)
- **elf_threshold**: Minimum ELF value for electride site detection (default: 0.5)